In [1]:
def create_runtime( runtime_name, runtime_main_function, filenames, imports ):

    import inspect
    lines_main_function = inspect.getsource(runtime_main_function)

    with open(runtime_name, 'w') as f:
        for iters in imports:
            f.write(iters+'\n')
        f.write('\n#####main runtime function\n')
        f.write(lines_main_function)

    remove_import = []
    remove_import_as = []

    for line in filenames:
        remove_import.append(  'from '+line[:-3]+' import *'  )
        remove_import.append(  'import '+line[:-3]  )
        remove_import_as.append(  'import '+line[:-3]+' as'  )

    imported_as = []

    with open(runtime_name, 'a') as outfile:
        for fname in filenames:
            outfile.write('\n#####'+fname+'\n')
            with open(fname) as infile:
                for line in infile:
                    if not any([ str_remove in line for str_remove in remove_import  ]) :
                        for str_remove in imported_as:
                            if str_remove+'.' in line:
                                line = line.replace(str_remove+'.','')
                        outfile.write(line) 
                    else :
                        for iter in remove_import_as:
                            if iter in line:
                                imported_as.append( line.strip()[-2:] )

In [2]:
filenames = [ 'main.py', 'QuantumTomography.py' ] 

In [3]:
imports = ['import numpy as np',
    'from qiskit import transpile',
    'from qiskit.providers.ibmq.managed import IBMQJobManager' ,
    'from qiskit.providers.aer.noise import NoiseModel' ]

In [4]:
def main( backend, user_messenger ):

    parall_qubits = [  [(0,1)]
                    ]

    qndmt = device_process_measurement_tomography( backend, parall_qubits=parall_qubits )
    circuits_qndmt = qndmt.circuits()

    # job_manager = IBMQJobManager()
    # job = job_manager.run( transpile( circuits_qndmt, backend ) , backend=backend, shots=2**13 )

    # job_id = job.job_set_id()
    
    # results = job.results().combine_results()
    
    n_circuits = len( circuits_qndmt )
    n_steps = 1 + n_circuits // 100
    circuit_split = n_circuits // n_steps
    jobs_id = []
    results = []
    
    for j in range(n_steps):
        job = backend.run( transpile( circuits_qndmt, backend ), shots=2**13  )
        jobs_id.append( job.job_id )
        
    results = job.result()

    choi_single, choi_double, gateset  = qndmt.fit( results, paralell=True, gate_set=True ) 

    dict_results = {  
                    'choi_single' : choi_single,
                    'choi_double' : choi_double,
                    'gateset' : gateset
                    }

    return dict_results

In [5]:
create_runtime( 'runtime.py', main, filenames, imports )

In [6]:
from runtime import main

In [7]:
from qiskit.test.mock import FakeVigo
from qiskit.providers.aer import AerSimulator
device_backend = FakeVigo()
sim_vigo = AerSimulator.from_backend( device_backend )

In [8]:
main(sim_vigo,[])

In [9]:
from qiskit import IBMQ

In [10]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

In [11]:
backend = provider.get_backend('ibm_nairobi')

In [12]:
meta = {
  "name": "QND measurement tomography",
  "description": "QND measurement tomography",
  "max_execution_time": 500000
}

meta["Input parameters"] = [
  ]

meta['Returns'] = [
    {"name": "result", "description": "Result in a dictionary.", "type": "dict"}
  ]

meta["Interim results"] = [

  ]

In [13]:
# provider.runtime.upload_program(data='runtime.py', metadata=meta)

In [14]:
program_id = 'qnd-measurement-tomography-z7MELYJmR7'
provider.runtime.update_program(data='runtime.py', metadata=meta, program_id=program_id)

In [15]:
prog = provider.runtime.program(program_id)
print(prog)

qnd-measurement-tomography-z7MELYJmR7:
  Name: QND measurement tomography
  Description: QND measurement tomography
  Creation date: 2022-08-05T17:47:59.701198Z
  Update date: 2022-08-09T09:14:04.741581Z
  Max execution time: 500000
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [16]:
options = {'backend_name': backend.name()}

In [17]:
inputs = { }

In [18]:
job = provider.runtime.run(program_id, options=options, inputs=inputs )
job.job_id()

'cbp2anbtlla56hssi9d0'